## FIFA Ranking Predictor

### Initialization

In [55]:
%matplotlib inline

from azureml import Workspace
import pandas as pd
import numpy as np

ws = Workspace()
ds = ws.datasets['fifa_ranking.csv']
fifa_df= ds.to_dataframe()

## Converting columns to numeric values
cols = ['rank_date']
fifa_df[cols] = fifa_df[cols].apply(pd.to_numeric, args=('coerce',))


##print (fifa_df.dtypes)
##fifa_df.head(5)

### Random Sampling

In [56]:
fifa_sam = fifa_df.sample(40000)

In [57]:
keep_col = [
    "total_points",
    "last_year_avg_weighted",
    "two_year_ago_weighted",
    "three_year_ago_weighted",
    "rank_date", "rank",
     ]

fifa_sam = fifa_sam[keep_col]

fifa_sam.columns= ["total_points",
    "last_year_avg_weighted",
    "two_year_ago_weighted",
    "three_year_ago_weighted",
    "rank_date","rank",]

In [58]:
fifa_sam.head(5)

,total_points,last_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,rank_date,rank
9950,0.00,0.00,0.00,0.00,1998,91
35110,0.00,0.00,0.00,0.00,2009,113
30306,0.00,0.00,0.00,0.00,2007,75
52327,958.79,259.03,167.34,120.53,2016,15
44009,434.85,237.15,12.45,30.96,2013,81


### Forming y Matrix


In [59]:
y = fifa_sam["rank"]

fifa_sam = fifa_sam.drop(labels=["rank"], axis=1)

y = np.reshape (y, (40000,1))


print(y.shape)


(40000, 1)


/home/nbuser/anaconda3_23/lib/python3.4/site-packages/numpy/core/fromnumeric.py:224: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return reshape(newshape, order=order)


In [60]:
m = len(fifa_sam)
theta = pd.DataFrame(columns=["theta"])

for index, value in enumerate(fifa_sam.columns.values):
    theta.loc[index] = 0

fifa_sam = fifa_sam.astype(np.float64)

In [61]:
print(theta.shape)
print(fifa_sam.shape)

fifa_sam.head(5)

(5, 1)
(40000, 5)


,total_points,last_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,rank_date
9950,0.00,0.00,0.00,0.00,1998.0
35110,0.00,0.00,0.00,0.00,2009.0
30306,0.00,0.00,0.00,0.00,2007.0
52327,958.79,259.03,167.34,120.53,2016.0
44009,434.85,237.15,12.45,30.96,2013.0


### Cost Function

In [62]:
def compute_cost(fifa_df, y, theta, m):
    h_x = np.dot(fifa_df, theta)
    h_x = pd.DataFrame(data=h_x)
    h_x = h_x[0]
    diff_val = h_x.subtract(y)
    square_val = diff_val ** 2
    sum_val = np.sum(square_val)
    J = sum_val / (2 * m)
    return J

### Normal Equation

In [63]:
def compute_normal(fifa_sam, y):
    norm_T = fifa_sam.T
    prod_df = norm_T.dot(fifa_sam)
    inv_df = pd.DataFrame(np.linalg.pinv(prod_df.values))
    new_df = inv_df.dot(norm_T.values)
    theta = new_df.dot(y)
    return theta
normal_theta = compute_normal(fifa_sam, y)

In [64]:
normal_theta

,0
0,-0.099104
1,0.030582
2,0.045496
3,0.069292
4,0.055518


In [65]:
 compute_cost(fifa_sam, y, normal_theta, m)

Exception: Data must be 1-dimensional